<Font size='6pt' color='blue'><center>RL_TP04</center></Font>

### 🎯 Objectif du TP  
L'objectif de ce TP est de **familiariser les étudiants** avec l’implémentation de l’algorithme **Proximal Policy Optimization (PPO)**, une méthode d’apprentissage par renforcement basée sur des politiques.

#### 📌 Contenu du TP  
1. 📊 **Initialisation de l'environement et des structures des donnes**  

2. 🚀 **Exploration et collecte d'episode**  

3. 🔄 **Mise a jour de La politique avec PPO**

4. **Evaluation de l'agent apres l'entrainement**


- **Importation des bibliotheques**

In [29]:
import gymnasium as gym
import numpy as np

- **Initialisation de l'environement et des structures des donnes**

In [30]:
# Initialisation de l'environnement
env = gym.make("Taxi-v3")

# Nombre d'états et d'actions
state_size = env.observation_space.n
action_size = env.action_space.n

# Table de politique (probabilités égales pour chaque action)
policy_table = np.full((state_size, action_size), 1 / action_size)

# Table de valeurs initialisée à zéro
value_table = np.zeros(state_size)

# Affichage des premières lignes
print("Policy table (extrait):\n", policy_table[:5])
print("\nValue table (extrait):\n", value_table[:5])


Policy table (extrait):
 [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Value table (extrait):
 [0. 0. 0. 0. 0.]


In [31]:
# Simulation d'un agent aléatoire sur 20 épisodes
for episode in range(20):
    state, _ = env.reset()  # Correction: env.reset() renvoie un tuple (state, info)
    done = False
    total_reward = 0

    print(f"\n=== Épisode {episode + 1} ===")
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, done, _, _ = env.step(action)

        print(f"État: {state}, Action: {action}, Récompense: {reward}")
        total_reward += reward
        state = next_state

    print(f"Récompense totale de l'épisode: {total_reward}")



=== Épisode 1 ===
État: 11, Action: 2, Récompense: -1
État: 31, Action: 4, Récompense: -10
État: 31, Action: 4, Récompense: -10
État: 31, Action: 5, Récompense: -10
État: 31, Action: 5, Récompense: -10
État: 31, Action: 3, Récompense: -1
État: 11, Action: 2, Récompense: -1
État: 31, Action: 1, Récompense: -1
État: 31, Action: 2, Récompense: -1
État: 31, Action: 3, Récompense: -1
État: 11, Action: 1, Récompense: -1
État: 11, Action: 0, Récompense: -1
État: 111, Action: 1, Récompense: -1
État: 11, Action: 3, Récompense: -1
État: 11, Action: 3, Récompense: -1
État: 11, Action: 0, Récompense: -1
État: 111, Action: 2, Récompense: -1
État: 131, Action: 0, Récompense: -1
État: 231, Action: 2, Récompense: -1
État: 251, Action: 5, Récompense: -10
État: 251, Action: 5, Récompense: -10
État: 251, Action: 0, Récompense: -1
État: 351, Action: 4, Récompense: -10
État: 351, Action: 3, Récompense: -1
État: 331, Action: 2, Récompense: -1
État: 351, Action: 1, Récompense: -1
État: 251, Action: 0, Récom

- **Mise a jour de la politique avec PPO**

In [32]:
# Hyperparamètres
gamma = 0.99          # Facteur de discount
lr_policy = 0.1       # Taux d'apprentissage pour la politique
clip_epsilon = 0.2    # Paramètre de clipping pour PPO

def update_ppo(episode_states, episode_actions, episode_rewards):
    # Calcul des récompenses cumulées (discounted rewards)
    discounted_rewards = []
    R = 0
    for r in reversed(episode_rewards):
        R = r + gamma * R
        discounted_rewards.insert(0, R)

    # Normalisation des récompenses
    discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)

    # Mise à jour de la politique et de la fonction de valeur
    for state, action, G in zip(episode_states, episode_actions, discounted_rewards):
        # Calcul de l'avantage
        advantage = G - value_table[state]

        # Mise à jour de la politique avec clipping
        old_prob = policy_table[state][action]
        new_prob = old_prob * np.exp(lr_policy * advantage)

        # Clipping pour éviter des mises à jour trop grandes
        ratio = new_prob / old_prob
        clipped_ratio = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon)
        policy_table[state][action] = old_prob + lr_policy * (clipped_ratio - 1) * advantage

        # Normalisation des probabilités
        policy_table[state] /= np.sum(policy_table[state])

        # Mise à jour de la fonction de valeur
        value_table[state] += lr_policy * advantage

# Exemple d'utilisation après un épisode
episode_states = [0, 1, 2]  # Exemple fictif
episode_actions = [1, 2, 0] # Exemple fictif
episode_rewards = [1, -1, 2] # Exemple fictif
update_ppo(episode_states, episode_actions, episode_rewards)



- **evaluation de model**

In [ ]:
def evaluate_agent(num_episodes=20):
    total_rewards = []
    for _ in range(num_episodes):
        state, _ = env.reset()  # Correction: env.reset() renvoie un tuple (state, info)
        done = False
        episode_reward = 0

        while not done:
            action = np.argmax(policy_table[state])  # Choix de l'action optimale
            state, reward, done, _, _ = env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)

    print(f"Récompenses moyennes sur {num_episodes} épisodes: {np.mean(total_rewards)}")
    return total_rewards

# Entraînement sur plusieurs épisodes (exemple simplifié)
for episode in range(100):  # 100 épisodes d'entraînement
    states, actions, rewards = [], [], []
    state, _ = env.reset()  # Correction: env.reset() renvoie un tuple (state, info)
    done = False

    while not done:
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, done, _, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        state = next_state

    update_ppo(states, actions, rewards)

# Évaluation finale
print("\n=== Performances après entraînement ===")
evaluate_agent()
